In [27]:
function plp(k, l) # poisson log probability
    k = convert(Int64, k)
    logprob = k*log(l) - l - log(factorial(k))
    return logprob
end

plp (generic function with 1 method)

In [28]:
## compute poisson probabilities in array?  
## one row each for 2, 3, and 4 (maybe have a 4*21 array, but don't bother with first line)

PoisProbs = zeros(Float64, 4, 21)

for row in 2:4
    PoisProbs[row,1] = 0.0
end

for row in 2:4
    sumprob = 0
    for col in 2:21
        prior = col - 2
        sumprob += exp(plp(prior,row))
        PoisProbs[row, col] = log(1 - sumprob)
    end
end

In [29]:
function compprob(bd, num, par)
    if num == bd
        par = convert(Int64, par)
        num = convert(Int64, num)
        return PoisProbs[par,num+1]
    else
        return plp(num, par)
    end
end

compprob (generic function with 1 method)

In [30]:
function p(end1, end2, rew, beg1, beg2, action)
    bs1a = beg1 - action
    bs2a = beg2 + action
    
    if (bs1a > 20)
        bs1a = 20
    end
    if (bs2a > 20)
        bs2a = 20
    end
    
    net1 = end1 - bs1a
    net2 = end2 - bs2a
    
    totreq = convert(Int64, rew/10) ## rew expected to be multiple of 10
    prob = 0
    for index = 0:totreq
        req1 = index
        req2 = totreq - index
        if (bs1a < req1) || (bs2a < req2)
            continue
        end
        ret1 = net1 + req1
        ret2 = net2 + req2
        if (ret1 < 0) || (ret2 < 0) || (req1 < 0) || (req2 < 0)
            continue
        end
        logprob = compprob(20 - bs1a + req1, ret1, 3) + compprob(20 - bs2a + req2, ret2, 2) + 
                  compprob(bs1a, req1, 3) + compprob(bs2a, req2, 4)
        prob += exp(logprob)
    end
    return prob
end

p (generic function with 1 method)

In [31]:
## start out with some constants
States = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
Actions = [-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5]
gamma = 0.9
lreq1 = 3
lret1 = 3
lreq2 = 4
lret2 = 2

2

In [32]:
PoisProbs ## REALLY logPoisProbs

4×21 Array{Float64,2}:
 0.0   0.0         0.0         0.0       …    0.0       0.0       0.0
 0.0  -0.145413   -0.520886   -1.1291       -25.8082  -28.0652  -30.372
 0.0  -0.0510692  -0.222079   -0.550242     -19.4503  -21.3052  -23.2104
 0.0  -0.0184854  -0.0960465  -0.271944     -15.2091  -16.78    -18.4008

In [33]:
Policy = zeros(Int8, 21, 21)
Values = zeros(Float64, 21, 21)

21×21 Array{Float64,2}:
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Now, code up Policy Evaluation and Policy Iteration.

In [42]:
## Policy Evaluation
delta = 10
theta = 0.01
while delta > theta
    for begstate1 = States
        for begstate2 = States
            v = Values[begstate1+1, begstate2+1]
            Values[begstate1+1, begstate2+1] = 0
            maxrew = (begstate1 + begstate2) * 10
            for estate1 = States
                for estate2 = States
                    for reward = 0:10:maxrew
                        Values[begstate1+1, begstate2+1] += p(estate1, estate2, 
                            reward, begstate1, begstate2, Policy[begstate1+1, begstate2+1]) * 
                            (reward - 2*abs(Policy[begstate1+1, begstate2+1]) + 
                            gamma * Values[estate1+1, estate2+1] )
                    end
                end
            end
            delta = max(theta, abs(v - Values[begstate1+1, begstate2+1]))
        end
    end
end

In [43]:
## Policy Improvement

policy_stable = true

for begstate1 = States
    for begstate2 = States
        old_action = Policy[begstate1+1, begstate2+1]
        actionvals = zeros(Float64, size(Actions))
        for index = 1:size(Actions)[1]
            action = Actions[index]
            if (begstate1 - action < 0) || (begstate2 + action < 0)
                continue ## don't allow to move more than you have
            end
            maxrew = (begstate1 + begstate2) * 10
            for estate1 = States
                for estate2 = States
                    for reward = 0:10:maxrew
                        actionvals[index] += p(estate1, estate2, 
                            reward, begstate1, begstate2, action) * 
                            (reward - 2*abs(action) + 
                            gamma * Values[estate1+1, estate2+1] )
                    end
                end
            end
        end
        Policy[begstate1+1, begstate2+1] = Actions[argmax(actionvals)]
        if old_action != Policy[begstate1+1, begstate2+1]
            policy_stable = false
        end
    end
end

In [44]:
for num = 1:21
    println(Policy[22-num,:])
end

Int8[5, 4, 4, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 0, 0, 0, 0]
Int8[5, 4, 3, 3, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0]
Int8[5, 4, 3, 2, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[5, 4, 3, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[4, 4, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[4, 3, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[4, 3, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[4, 3, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[3, 3, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[3, 2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[3, 2, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[2, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Int8[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0